<a href="https://colab.research.google.com/github/Nscaglio/4105/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PART 1**

In [10]:
import torch
import torch.optim as optim
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)
#1.a
def model(t_u, w1, w2 , b):
  return w2 * t_u ** 2 + w1 * t_u + b
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
  optimizer = optim.Adam([params], lr=learning_rate)
  for epoch in range(1, n_epochs + 1):
        optimizer.zero_grad()
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print(f'Epoch {epoch}, Loss {float(loss):.6f}')

  return params
params1 = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
params2 = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
params3 = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
params4 = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
#1.b
# Train the model
trained_params1 = training_loop(n_epochs=5000, learning_rate=0.1, params=params1, t_u=t_u, t_c=t_c)

Epoch 500, Loss 4.844501
Epoch 1000, Loss 3.785644
Epoch 1500, Loss 3.466676
Epoch 2000, Loss 3.406405
Epoch 2500, Loss 3.377975
Epoch 3000, Loss 3.344335
Epoch 3500, Loss 3.301921
Epoch 4000, Loss 3.249141
Epoch 4500, Loss 3.184418
Epoch 5000, Loss 3.106353


In [2]:
trained_params2= training_loop(n_epochs=5000, learning_rate=0.01, params=params2, t_u=t_u, t_c=t_c)

Epoch 500, Loss 6.108693
Epoch 1000, Loss 5.984358
Epoch 1500, Loss 5.809032
Epoch 2000, Loss 5.586317
Epoch 2500, Loss 5.320902
Epoch 3000, Loss 5.021267
Epoch 3500, Loss 4.701309
Epoch 4000, Loss 4.380754
Epoch 4500, Loss 4.083409
Epoch 5000, Loss 3.832656


In [3]:
trained_params3= training_loop(n_epochs=5000, learning_rate=0.001, params=params3, t_u=t_u, t_c=t_c)

Epoch 500, Loss 3668163.000000
Epoch 1000, Loss 768786.437500
Epoch 1500, Loss 90021.125000
Epoch 2000, Loss 4791.339844
Epoch 2500, Loss 98.014191
Epoch 3000, Loss 6.609710
Epoch 3500, Loss 6.117270
Epoch 4000, Loss 6.096753
Epoch 4500, Loss 6.071090
Epoch 5000, Loss 6.038404


In [4]:
trained_params4= training_loop(n_epochs=5000, learning_rate=0.0001, params=params4, t_u=t_u, t_c=t_c)

Epoch 500, Loss 10577728.000000
Epoch 1000, Loss 9524402.000000
Epoch 1500, Loss 8545122.000000
Epoch 2000, Loss 7634292.500000
Epoch 2500, Loss 6787368.000000
Epoch 3000, Loss 6000706.000000
Epoch 3500, Loss 5271407.500000
Epoch 4000, Loss 4597170.000000
Epoch 4500, Loss 3976134.250000
Epoch 5000, Loss 3406753.750000


It appears that the learning rate of 0.1 did the best as it converged at a reasonable rate.


Part 2

In [5]:
#set up
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ECGR4105/Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})
housing[varlist] = housing[varlist].apply(binary_map)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X1 = housing[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']]
y = housing['price']
# Normalize the features (X1)
scaler = StandardScaler()
# Fit the scaler on the training data and transform both the training and validation data
X1 = scaler.fit_transform(X1)  # Normalization
X_train, X_val, y_train, y_val = train_test_split(X1, y, test_size=0.2, random_state=42)

# Convert back to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)


y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

In [51]:
def model(t_u, w, b):
  return torch.matmul(t_u, w) + b  # Linear combination of features + bias
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()
def training_loop(n_epochs, learning_rate, w, b, t_u_train, t_c_train, t_u_val, t_c_val):
    # Set up Adam optimizer
    optimizer = optim.Adam([w, b], lr=learning_rate)
    for epoch in range(1, n_epochs + 1):
        optimizer.zero_grad()  # Clear previous gradients
        # Forward pass on training data
        t_p_train = model(t_u_train, w, b)
        # Compute the training loss
        train_loss = loss_fn(t_p_train, t_c_train)
        # Backpropagation: compute gradients
        train_loss.backward()
        # Parameter update: adjust parameters using Adam
        optimizer.step()
        # Compute the validation loss
        with torch.no_grad():  # No need to compute gradients during evaluation
            t_p_val = model(t_u_val, w, b)
            val_loss = loss_fn(t_p_val, t_c_val)
        # Print loss every 500 epochs
        if epoch % 500 == 0:
            print(f'Epoch {epoch}, Train Loss {float(train_loss):.6f}, Val Loss {float(val_loss):.6f}')
    return w, b
# Initialize parameters: weights (w) and bias (b)
num_features = X_train.shape[1]  # Number of input features
w1 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b1 = torch.randn(1, requires_grad=True)  # Random initialization of bias

# Training the model using the training data (X_train, y_train)
n_epochs = 5000
learning_rate = 0.1
w1, b1 = training_loop(n_epochs, learning_rate, w1, b1, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25233981440000.000000, Val Loss 30128813899776.000000
Epoch 1000, Train Loss 25233169842176.000000, Val Loss 30127624814592.000000
Epoch 1500, Train Loss 25232349855744.000000, Val Loss 30126435729408.000000
Epoch 2000, Train Loss 25231531966464.000000, Val Loss 30125242449920.000000
Epoch 2500, Train Loss 25230711980032.000000, Val Loss 30124051267584.000000
Epoch 3000, Train Loss 25229894090752.000000, Val Loss 30122857988096.000000
Epoch 3500, Train Loss 25229078298624.000000, Val Loss 30121668902912.000000
Epoch 4000, Train Loss 25228260409344.000000, Val Loss 30120479817728.000000
Epoch 4500, Train Loss 25227442520064.000000, Val Loss 30119290732544.000000
Epoch 5000, Train Loss 25226626727936.000000, Val Loss 30118099550208.000000


In [36]:
learning_rate = 0.01
w2 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b2 = torch.randn(1, requires_grad=True)  # Random initialization of bias
w2, b2 = training_loop(n_epochs, learning_rate, w2, b2, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25234713346048.000000, Val Loss 30129879252992.000000
Epoch 1000, Train Loss 25234631557120.000000, Val Loss 30129759715328.000000
Epoch 1500, Train Loss 25234547671040.000000, Val Loss 30129644371968.000000
Epoch 2000, Train Loss 25234470076416.000000, Val Loss 30129520640000.000000
Epoch 2500, Train Loss 25234388287488.000000, Val Loss 30129403199488.000000
Epoch 3000, Train Loss 25234304401408.000000, Val Loss 30129281564672.000000
Epoch 3500, Train Loss 25234222612480.000000, Val Loss 30129164124160.000000
Epoch 4000, Train Loss 25234145017856.000000, Val Loss 30129050877952.000000
Epoch 4500, Train Loss 25234061131776.000000, Val Loss 30128929243136.000000
Epoch 5000, Train Loss 25233979342848.000000, Val Loss 30128809705472.000000


In [38]:
learning_rate = 0.0001
w4 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b4 = torch.randn(1, requires_grad=True)  # Random initialization of bias
w4, b4 = training_loop(n_epochs, learning_rate, w4, b4, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25234788843520.000000, Val Loss 30129990402048.000000
Epoch 1000, Train Loss 25234788843520.000000, Val Loss 30129990402048.000000
Epoch 1500, Train Loss 25234786746368.000000, Val Loss 30129988304896.000000
Epoch 2000, Train Loss 25234786746368.000000, Val Loss 30129988304896.000000
Epoch 2500, Train Loss 25234786746368.000000, Val Loss 30129986207744.000000
Epoch 3000, Train Loss 25234784649216.000000, Val Loss 30129986207744.000000
Epoch 3500, Train Loss 25234784649216.000000, Val Loss 30129986207744.000000
Epoch 4000, Train Loss 25234784649216.000000, Val Loss 30129984110592.000000
Epoch 4500, Train Loss 25234784649216.000000, Val Loss 30129982013440.000000
Epoch 5000, Train Loss 25234784649216.000000, Val Loss 30129982013440.000000


In [37]:
learning_rate = 0.001
w3 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b3 = torch.randn(1, requires_grad=True)  # Random initialization of bias
w3, b3 = training_loop(n_epochs, learning_rate, w3, b3, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25234782552064.000000, Val Loss 30129984110592.000000
Epoch 1000, Train Loss 25234774163456.000000, Val Loss 30129967333376.000000
Epoch 1500, Train Loss 25234767872000.000000, Val Loss 30129958944768.000000
Epoch 2000, Train Loss 25234755289088.000000, Val Loss 30129944264704.000000
Epoch 2500, Train Loss 25234751094784.000000, Val Loss 30129933778944.000000
Epoch 3000, Train Loss 25234740609024.000000, Val Loss 30129919098880.000000
Epoch 3500, Train Loss 25234730123264.000000, Val Loss 30129906515968.000000
Epoch 4000, Train Loss 25234725928960.000000, Val Loss 30129896030208.000000
Epoch 4500, Train Loss 25234715443200.000000, Val Loss 30129887641600.000000
Epoch 5000, Train Loss 25234709151744.000000, Val Loss 30129872961536.000000


Again the high learning rate is the best for the regression because it converges the quickest, it's just that the loss values are massive.
generally it did better than homework 1 where the loss was just NaN

**Part 3**

In [56]:
def trinary_map(x):
    mapping = {'furnished': 1, 'semi-furnished': 0.5, 'unfurnished': 0}
    return mapping.get(x, 0)  # Default to 0 if the value is not in the mapping
housing['furnishingstatus'] = housing['furnishingstatus'].apply(trinary_map)
X2 = housing[['area',	'bedrooms',	'bathrooms',	'stories',	'mainroad',	'guestroom',	'basement',	'hotwaterheating',	'airconditioning',	'parking',	'prefarea','furnishingstatus']]
y = housing['price']
X2 = scaler.fit_transform(X2)  # Normalization
X_train, X_val, y_train, y_val = train_test_split(X1, y, test_size=0.2, random_state=42)
# Convert back to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

In [58]:
num_features = X_train.shape[1]  # Number of input features
w1 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b1 = torch.randn(1, requires_grad=True)  # Random initialization of bias

# Training the model using the training data (X_train, y_train)
learning_rate = 0.1
w1, b1 = training_loop(n_epochs, learning_rate, w1, b1, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25233983537152.000000, Val Loss 30128811802624.000000
Epoch 1000, Train Loss 25233163550720.000000, Val Loss 30127618523136.000000
Epoch 1500, Train Loss 25232349855744.000000, Val Loss 30126425243648.000000
Epoch 2000, Train Loss 25231531966464.000000, Val Loss 30125238255616.000000
Epoch 2500, Train Loss 25230711980032.000000, Val Loss 30124049170432.000000
Epoch 3000, Train Loss 25229894090752.000000, Val Loss 30122855890944.000000
Epoch 3500, Train Loss 25229080395776.000000, Val Loss 30121662611456.000000
Epoch 4000, Train Loss 25228260409344.000000, Val Loss 30120475623424.000000
Epoch 4500, Train Loss 25227442520064.000000, Val Loss 30119280246784.000000
Epoch 5000, Train Loss 25226626727936.000000, Val Loss 30118093258752.000000


In [60]:
w2 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b2 = torch.randn(1, requires_grad=True)  # Random initialization of bias

# Training the model using the training data (X_train, y_train)
learning_rate = 0.01
w2, b2 = training_loop(n_epochs, learning_rate, w2, b2, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25234694471680.000000, Val Loss 30129854087168.000000
Epoch 1000, Train Loss 25234610585600.000000, Val Loss 30129732452352.000000
Epoch 1500, Train Loss 25234528796672.000000, Val Loss 30129610817536.000000
Epoch 2000, Train Loss 25234449104896.000000, Val Loss 30129493377024.000000
Epoch 2500, Train Loss 25234365218816.000000, Val Loss 30129375936512.000000
Epoch 3000, Train Loss 25234285527040.000000, Val Loss 30129256398848.000000
Epoch 3500, Train Loss 25234201640960.000000, Val Loss 30129136861184.000000
Epoch 4000, Train Loss 25234119852032.000000, Val Loss 30129015226368.000000
Epoch 4500, Train Loss 25234038063104.000000, Val Loss 30128897785856.000000
Epoch 5000, Train Loss 25233954177024.000000, Val Loss 30128782442496.000000


In [61]:
w3 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b3 = torch.randn(1, requires_grad=True)  # Random initialization of bias

# Training the model using the training data (X_train, y_train)
learning_rate = 0.001
w3, b3 = training_loop(n_epochs, learning_rate, w3, b3, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25234774163456.000000, Val Loss 30129971527680.000000
Epoch 1000, Train Loss 25234765774848.000000, Val Loss 30129963139072.000000
Epoch 1500, Train Loss 25234759483392.000000, Val Loss 30129952653312.000000
Epoch 2000, Train Loss 25234751094784.000000, Val Loss 30129935876096.000000
Epoch 2500, Train Loss 25234742706176.000000, Val Loss 30129927487488.000000
Epoch 3000, Train Loss 25234736414720.000000, Val Loss 30129912807424.000000
Epoch 3500, Train Loss 25234725928960.000000, Val Loss 30129902321664.000000
Epoch 4000, Train Loss 25234721734656.000000, Val Loss 30129891835904.000000
Epoch 4500, Train Loss 25234711248896.000000, Val Loss 30129875058688.000000
Epoch 5000, Train Loss 25234700763136.000000, Val Loss 30129862475776.000000


In [62]:
w4 = torch.randn(num_features, requires_grad=True)  # Random initialization of weights
b4 = torch.randn(1, requires_grad=True)  # Random initialization of bias

# Training the model using the training data (X_train, y_train)
learning_rate = 0.0001
w4, b4 = training_loop(n_epochs, learning_rate, w4, b4, X_train, y_train, X_val, y_val)

Epoch 500, Train Loss 25234780454912.000000, Val Loss 30129973624832.000000
Epoch 1000, Train Loss 25234774163456.000000, Val Loss 30129971527680.000000
Epoch 1500, Train Loss 25234774163456.000000, Val Loss 30129973624832.000000
Epoch 2000, Train Loss 25234774163456.000000, Val Loss 30129969430528.000000
Epoch 2500, Train Loss 25234774163456.000000, Val Loss 30129967333376.000000
Epoch 3000, Train Loss 25234774163456.000000, Val Loss 30129967333376.000000
Epoch 3500, Train Loss 25234772066304.000000, Val Loss 30129963139072.000000
Epoch 4000, Train Loss 25234772066304.000000, Val Loss 30129963139072.000000
Epoch 4500, Train Loss 25234772066304.000000, Val Loss 30129961041920.000000
Epoch 5000, Train Loss 25234772066304.000000, Val Loss 30129961041920.000000


Again the high learning rate is the best for the regression because it converges the quickest, it's just that the loss values are massive. it did not do much any better or worse than with less features.